# Pre-Processing

In this notebook we will use what we learned in lab 03 about forming the data, the end result is a dataset that we can feed weka to test models on.

### So we will:

1. [Load the dataset](#chapter1)
2. [Split the text into sentences](#chapter2)
3. [Perform sentiment analysis on the sentences](#chapter3)
4. [Include statistical features](#chapter4)
5. [Resample the data to 150 points](#chapter5)
6. [Store it in a row of a dataframe](#chapter6)
7. [Export it to a csv](#chapter7)

In [ ]:
# imports
import os
import pandas as pd
import numpy as np

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()

import nltk
#file for punkt splitter
nltk.download('punkt');
#file for vader sentiment
nltk.download('vader_lexicon');

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=20,20
%matplotlib inline

<br>

### 1: Load the dataset <a class="anchor" id="chapter1"></a>

In [ ]:
dataset_path_true = os.path.join("sources", "ISOT", "True.csv")
dataset_path_fake = os.path.join("sources", "ISOT", "Fake.csv")

dataset_load_true = pd.read_csv(dataset_path_true, encoding='utf-8') # make sure to use the right encoding
dataset_load_fake = pd.read_csv(dataset_path_fake, encoding='utf-8') 

display(dataset_load_true.head())
display(dataset_load_fake.head())

<br>

### 2: Split the text into sentences <a class="anchor" id="chapter2"></a>

in lab 03 we used a big manual function, but for now we will use the nltk tokenizer in this one :)

In [ ]:
def split_sentences(article_text):
    """Takes a string, returns a list of its individual sentences ()"""
    return pd.Series(nltk.tokenize.sent_tokenize(article_text))

sample_sentences = split_sentences(dataset_load_true.text[0])
display(sample_sentences.iloc[[0,1,-2,-1]])

<br>

### 3: Performing sentiment analysis <a class="anchor" id="chapter3"></a>


In [ ]:
def get_scores(text: list, method='VADER'):
    if method == 'VADER':
        scores = text.apply(lambda s: sia.polarity_scores(s)['compound']) #list of compound score per sentence
    else:
        scores = None

    return scores

sample_scores = get_scores(sample_sentences)
print(sample_scores.iloc[[0,1,-2,-1]])

<br>

### 4: Include statistical features <a class="anchor" id="chapter4"></a>


In [ ]:
def get_stats(scores):
    stats = scores.describe()
    return stats

sample_stats = get_stats(sample_scores)
print(sample_stats)

<br>

### 5: Resample the data <a class="anchor" id="chapter5"></a>



In [ ]:
def resample_to_x(scores, scores_len, x = 200, to_dict = False):
    if scores_len <= x: #resample to higher (or same) res
        if scores_len > 1:
            slope = (x - 1)/ (scores_len - 1)
            new_indices = ['p_'+f'{int(slope * i):03}' for i in range(int(scores_len))]

            new_scores = scores[:]
            new_scores.index = new_indices

            new_series = pd.Series(index = ['p_'+f'{i:03}' for i in range(int(x))], dtype='float64')

            output = new_scores.combine_first(new_series)
            output = output.interpolate()
        elif scores_len == 1:
            output = pd.Series(data = scores[0], index = ['p_'+f'{i:03}' for i in range(int(x))], dtype='float64')
        else:
            output = pd.Series(data = None, index = ['p_'+f'{i:03}' for i in range(int(x))], dtype='float64')
            
    elif scores_len > x: #downscale ;(
        y = int(np.ceil(len(scores)/x)) # bin size 
        
        new_scores = [np.mean(i) for i in np.array_split(np.array(scores), x)]
        output = pd.Series(data = new_scores, index = ['p_'+f'{i:03}' for i in range(int(x))], dtype='float64')
        
    if not to_dict:
        return output
    else:
        return output.to_dict()
    
sample_interp = resample_to_x(sample_scores, sample_stats['count'])


In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1)
plt.subplots_adjust(hspace=0.8)

sample_scores.plot(ax=axes[0]);
axes[0].set_title('Original Scores');

sample_interp.plot(ax=axes[1]);
axes[1].set_title('Resampled Scores');

downsample_interp = resample_to_x(sample_scores, sample_stats['count'], x=10)
downsample_interp.plot(ax=axes[2]);
axes[2].set_title('Downsampled Scores');


<br>

### 6: Create the dataframe <a class="anchor" id="chapter6"></a>

In [ ]:
def get_all(text, veracity: int, resample_size: int = 200):
    try: 
        # should all be dicts with the appropriate name
        sentences = split_sentences(text) # list
        scores = get_scores(sentences) #  list but ok, will be
        stats = get_stats(scores) # dict yes!
        scores_interp = resample_to_x(scores, stats['count'], resample_size, to_dict=True) # dict si (p_000 through p_resamplesize)
        v = {'veracity': veracity}
        v.update(stats)
        v.update(scores_interp)
        return pd.Series(v)
    except Exception as e:
        print("ERROR on text: \n"+text+"\n\n",stats['count'])
        raise e

# print(get_all(dataset_load_true.text[0], 1))

In [ ]:
%%time

# this cell is for verifying that the method at least works on a small scale (8 instances)
# much quicker to debug than large scale (40k+ instances) ;(

resample_size = 10

cols = ['veracity', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
cols.extend(['p_'+f'{n:03}' for n in range(resample_size)])

miniset_true = dataset_load_true.head()
miniset_fake = dataset_load_fake.head()

miniset_prepro = pd.DataFrame(columns = cols)
miniset_prepro[cols] = miniset_true.text.apply(get_all, args=(1, resample_size));
miniset_prepro = pd.concat([miniset_prepro, (miniset_fake.text.apply(get_all, args=(0, resample_size)))], ignore_index=True)

miniset_prepro.iloc[[0,1,-2,-1]]

In [ ]:
%%time

resample_size = 200

cols = ['veracity', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
cols.extend(['p_'+f'{n:03}' for n in range(resample_size)])

dataset_prepro = pd.DataFrame(columns = cols)

print("We'll give it a shot!")
dataset_prepro[cols] = dataset_load_true.text.apply(get_all, args=(1, resample_size));
print("Woah, we're halfway there!")
dataset_prepro = pd.concat([miniset_prepro, (dataset_load_fake.text.apply(get_all, args=(0, resample_size)))], ignore_index=True);
print("Woah, livin' on a prayer")
